# LSTM для генерации музыки

In [2]:
import os
import pickle
import numpy
from music21 import note, chord

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import plot_model

from new.RNNAttention import get_distinct, create_lookups, prepare_sequences, get_music_list, create_network

## Установка параметров

In [3]:
# run params
section = 'compose'
run_id = '0006'
music_name = 'cello'

run_folder = 'run/{}/'.format(section)
run_folder += '_'.join([run_id, music_name])


store_folder = os.path.join(run_folder, 'store')
data_folder = os.path.join('data', music_name)

if not os.path.exists(run_folder):
    os.mkdir(run_folder)
    os.mkdir(os.path.join(run_folder, 'store'))
    os.mkdir(os.path.join(run_folder, 'output'))
    os.mkdir(os.path.join(run_folder, 'weights'))
    os.mkdir(os.path.join(run_folder, 'viz'))
    


mode = 'build' # 'load' # 

# параметры данных
intervals = range(1)
seq_len = 32

# параметры модели
embed_size = 100
rnn_units = 256
use_attention = True

## Подготовка данных

In [4]:
if mode == 'build':
    
    music_list, parser = get_music_list(data_folder)
    print(len(music_list), 'files in total')

    notes = []
    durations = []

    for i, file in enumerate(music_list):
        print(i+1, "Parsing %s" % file)
        original_score = parser.parse(file).chordify()
        

        for interval in intervals:

            score = original_score.transpose(interval)

            notes.extend(['START'] * seq_len)
            durations.extend([0]* seq_len)

            for element in score.flat:
                
                if isinstance(element, note.Note):
                    if element.isRest:
                        notes.append(str(element.name))
                        durations.append(element.duration.quarterLength)
                    else:
                        notes.append(str(element.nameWithOctave))
                        durations.append(element.duration.quarterLength)

                if isinstance(element, chord.Chord):
                    notes.append('.'.join(n.nameWithOctave for n in element.pitches))
                    durations.append(element.duration.quarterLength)

    with open(os.path.join(store_folder, 'notes'), 'wb') as f:
        pickle.dump(notes, f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, 'durations'), 'wb') as f:
        pickle.dump(durations, f) 
else:
    with open(os.path.join(store_folder, 'notes'), 'rb') as f:
        notes = pickle.load(f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, 'durations'), 'rb') as f:
        durations = pickle.load(f) 

36 files in total
1 Parsing data\cello\cs1-1pre.mid
2 Parsing data\cello\cs1-2all.mid
3 Parsing data\cello\cs1-3cou.mid
4 Parsing data\cello\cs1-4sar.mid
5 Parsing data\cello\cs1-5men.mid
6 Parsing data\cello\cs1-6gig.mid
7 Parsing data\cello\cs2-1pre.mid
8 Parsing data\cello\cs2-2all.mid
9 Parsing data\cello\cs2-3cou.mid
10 Parsing data\cello\cs2-4sar.mid
11 Parsing data\cello\cs2-5men.mid
12 Parsing data\cello\cs2-6gig.mid
13 Parsing data\cello\cs3-1pre.mid
14 Parsing data\cello\cs3-2all.mid
15 Parsing data\cello\cs3-3cou.mid
16 Parsing data\cello\cs3-4sar.mid
17 Parsing data\cello\cs3-5bou.mid
18 Parsing data\cello\cs3-6gig.mid
19 Parsing data\cello\cs4-1pre.mid
20 Parsing data\cello\cs4-2all.mid
21 Parsing data\cello\cs4-3cou.mid
22 Parsing data\cello\cs4-4sar.mid
23 Parsing data\cello\cs4-5bou.mid
24 Parsing data\cello\cs4-6gig.mid
25 Parsing data\cello\cs5-1pre.mid
26 Parsing data\cello\cs5-2all.mid
27 Parsing data\cello\cs5-3cou.mid
28 Parsing data\cello\cs5-4sar.mid
29 Parsing 

## создание таблиц

In [7]:
# получаем наборы нот и длительностей
note_names, n_notes = get_distinct(notes)
duration_names, n_durations = get_distinct(durations)
distincts = [note_names, n_notes, duration_names, n_durations]

with open(os.path.join(store_folder, 'distincts'), 'wb') as f:
    pickle.dump(distincts, f)

# созадние словарей
note_to_int, int_to_note = create_lookups(note_names)
duration_to_int, int_to_duration = create_lookups(duration_names)
lookups = [note_to_int, int_to_note, duration_to_int, int_to_duration]

with open(os.path.join(store_folder, 'lookups'), 'wb') as f:
    pickle.dump(lookups, f)

In [8]:
print('\nnote_to_int')
note_to_int


note_to_int


{'A2': 0,
 'A2.A3': 1,
 'A2.B-3': 2,
 'A2.B2': 3,
 'A2.C3': 4,
 'A2.D3': 5,
 'A2.E-3': 6,
 'A2.E3': 7,
 'A2.E3.A3': 8,
 'A2.E3.C#4': 9,
 'A2.E3.C#4.A4': 10,
 'A2.E3.C#4.E4': 11,
 'A2.E3.C#4.G#4': 12,
 'A2.E3.C4': 13,
 'A2.E3.D4': 14,
 'A2.F#3': 15,
 'A2.F#3.C4': 16,
 'A2.F#3.D4': 17,
 'A2.F#3.D4.A4': 18,
 'A2.F#3.D4.E4': 19,
 'A2.F#3.D4.F#4': 20,
 'A2.F#4': 21,
 'A2.F3': 22,
 'A2.F3.C4': 23,
 'A2.F3.D4': 24,
 'A2.F3.D4.A4': 25,
 'A2.G3': 26,
 'A2.G3.C#4': 27,
 'A2.G3.D4': 28,
 'A3': 29,
 'A3.B-3': 30,
 'A3.B3': 31,
 'A3.B3.C#4': 32,
 'A3.B3.C#4.D4': 33,
 'A3.B3.C4': 34,
 'A3.B3.D4': 35,
 'A3.C#4': 36,
 'A3.C#4.E4': 37,
 'A3.C4': 38,
 'A3.D4': 39,
 'A3.E4': 40,
 'A3.E4.F#4': 41,
 'A3.E4.G4': 42,
 'A3.F#4': 43,
 'A3.F#4.G4': 44,
 'A3.F4': 45,
 'A3.G4': 46,
 'A4': 47,
 'A4.B4': 48,
 'A4.C5': 49,
 'B-2': 50,
 'B-2.A3': 51,
 'B-2.B-3': 52,
 'B-2.D3': 53,
 'B-2.D3.A3': 54,
 'B-2.D3.E-3.G#3': 55,
 'B-2.D3.G#3': 56,
 'B-2.E-3': 57,
 'B-2.E3': 58,
 'B-2.E3.D4': 59,
 'B-2.F#3.C#4.E4': 60,
 'B-2.

In [9]:
print('\nduration_to_int')
duration_to_int


duration_to_int


{0: 0,
 Fraction(1, 12): 1,
 Fraction(1, 6): 2,
 0.25: 3,
 Fraction(1, 3): 4,
 Fraction(5, 12): 5,
 0.5: 6,
 Fraction(2, 3): 7,
 0.75: 8,
 1.0: 9,
 1.25: 10,
 Fraction(4, 3): 11,
 1.5: 12,
 1.75: 13,
 2.0: 14,
 2.25: 15,
 2.5: 16,
 3.0: 17,
 4.0: 18}

## Подготовка последовательностей

In [10]:
network_input, network_output = prepare_sequences(notes, durations, lookups, distincts, seq_len)

In [11]:
print('pitch input')
print(network_input[0][0])
print('duration input')
print(network_input[1][0])
print('pitch output')
print(network_output[0][0])
print('duration output')
print(network_output[1][0])

pitch input
[473 473 473 473 473 473 473 473 473 473 473 473 473 473 473 473 473 473
 473 473 473 473 473 473 473 473 473 473 473 473 473 473]
duration input
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
pitch output
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

## Модель сети

In [12]:
model, att_model = create_network(n_notes, n_durations, embed_size, rnn_units, use_attention)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 100)    47400       input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    1900        input_2[0][0]                    
______________________________________________________________________________________________

C:\Users\geliron\anaconda3\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


## Тренировка сети

In [14]:
weights_folder = os.path.join(run_folder, 'weights')
# model.load_weights(os.path.join(weights_folder, "weights.h5"))

In [16]:
weights_folder = os.path.join(run_folder, 'weights')

checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1
    , checkpoint2
    , early_stopping
 ]

model.save_weights(os.path.join(weights_folder, "weights.h5"))
model.fit(network_input, network_output
          , epochs=5000, batch_size=32
          , validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )



Epoch 1/5000
729/729 [==============================] - 16s 22ms/step - loss: 3.5995 - pitch_loss: 3.0303 - duration_loss: 0.5692 - val_loss: 5.8297 - val_pitch_loss: 5.0067 - val_duration_loss: 0.8230
Epoch 2/5000
729/729 [==============================] - 16s 22ms/step - loss: 3.4098 - pitch_loss: 2.8927 - duration_loss: 0.5171 - val_loss: 5.7974 - val_pitch_loss: 5.0037 - val_duration_loss: 0.7938
Epoch 3/5000
729/729 [==============================] - 16s 21ms/step - loss: 3.2434 - pitch_loss: 2.7739 - duration_loss: 0.4695 - val_loss: 5.7993 - val_pitch_loss: 4.9660 - val_duration_loss: 0.8333
Epoch 4/5000
729/729 [==============================] - 18s 24ms/step - loss: 3.0930 - pitch_loss: 2.6674 - duration_loss: 0.4255 - val_loss: 6.1156 - val_pitch_loss: 5.2361 - val_duration_loss: 0.8795
Epoch 5/5000
729/729 [==============================] - 17s 23ms/step - loss: 2.9399 - pitch_loss: 2.5534 - duration_loss: 0.3865 - val_loss: 5.7581 - val_pitch_loss: 4.8572 - val_duration_los

729/729 [==============================] - 17s 24ms/step - loss: 0.1836 - pitch_loss: 0.1602 - duration_loss: 0.0234 - val_loss: 12.8539 - val_pitch_loss: 10.0136 - val_duration_loss: 2.8403
Epoch 82/5000
729/729 [==============================] - 17s 23ms/step - loss: 0.1867 - pitch_loss: 0.1615 - duration_loss: 0.0252 - val_loss: 12.7148 - val_pitch_loss: 9.9573 - val_duration_loss: 2.7575
Epoch 83/5000
729/729 [==============================] - 17s 23ms/step - loss: 0.1823 - pitch_loss: 0.1564 - duration_loss: 0.0259 - val_loss: 12.7594 - val_pitch_loss: 9.9914 - val_duration_loss: 2.7680
Epoch 84/5000
729/729 [==============================] - 17s 23ms/step - loss: 0.1797 - pitch_loss: 0.1545 - duration_loss: 0.0252 - val_loss: 12.7212 - val_pitch_loss: 10.0335 - val_duration_loss: 2.6877
Epoch 85/5000
729/729 [==============================] - 17s 24ms/step - loss: 0.1760 - pitch_loss: 0.1517 - duration_loss: 0.0243 - val_loss: 12.8443 - val_pitch_loss: 10.1013 - val_duration_loss